In [3]:
from jinja2 import Template
from pathlib import Path
import argparse
import json
import openai
import os
from datetime import datetime
import keyring
import uuid

In [1]:
import jinja2
from jinja2 import Template

def _TS(TEMPLATE_INPUT, args={}):
    # Always use FileSystemLoader with the current directory
    templateLoader = jinja2.FileSystemLoader(searchpath="./")
    templateEnv = jinja2.Environment(loader=templateLoader)
    
    # Check if the input is a file in the current directory
    if os.path.isfile(TEMPLATE_INPUT):
        # Load from a file
        template = templateEnv.get_template(TEMPLATE_INPUT)
    else:
        # If not a file, load from a string
        # However, the FileSystemLoader is still in effect for any imported/included templates
        template = Template(TEMPLATE_INPUT)
        template.environment = templateEnv

    return template.render(**args)


In [4]:
openai.api_key =keyring.get_password("system", "openai_key")

In [12]:
class ChatRoom:
    def __init__(self, room_id=None,model='gpt-3.5-turbo',system_prompt="You are a helpful assistant.",save=True):
        if room_id is None:
            room_id = uuid.uuid4()
        self.room_id = room_id
        self.chat_history_file = f'{room_id}_chat_history.json'
        self.model = model
        self.system_prompt = system_prompt
        self.save = save
        
        self.chat_history = []
        if save:
            if not os.path.exists(self.chat_history_file):
                with open(self.chat_history_file, 'w') as f:
                    json.dump([], f)
            else:
                self.get_chat_history()
            
    def get_chat_history(self):
        if self.save:
            with open(self.chat_history_file, 'r') as f:
                self.chat_history = json.load(f)

    def save_chat_history(self):
        if self.save:
            with open(self.chat_history_file, 'w') as f:
                json.dump(self.chat_history, f)
            
    def clone(self):
        new_room_id = f"{self.room_id}_{uuid.uuid4()}"
        new_chatroom = ChatRoom(new_room_id, model=self.model, system_prompt=self.system_prompt,save=self.save)
        
        # Copy chat history
        new_chatroom.chat_history = self.chat_history
        new_chatroom.save_chat_history()
        
        return new_chatroom
    
    def send_message(self, message,respond = True,max_tokens=2000,temperature=0.7):
        self.chat_history.append({
            'content': message,
            'role': 'user'
        })

        if respond:
            # Prepare the messages for the API call.
            api_messages = [{"role": "system", "content": self.system_prompt}]

            # Accurately estimate the token count using Tiktoken and trim messages accordingly.
            token_count = 0
            for msg in reversed(self.chat_history):
                api_messages.insert(1,msg)  # Insert each message at the start to keep the order.

            self.last_response = openai.ChatCompletion.create(
                model=self.model,
                messages=api_messages,
                temperature=temperature,
                max_tokens=max_tokens 
            ).choices[0].message['content']

            self.chat_history.append({
                'content': self.last_response,
                'role': 'assistant'
            })
        self.save_chat_history()

        if respond:
            return self.last_response
        else:
            return True

In [13]:
parser = argparse.ArgumentParser(description="Process a file.")
parser.add_argument('--file_path', type=str, default="Prompt.AiPrompt", help="Path to the file to process")
args = parser.parse_args()
file_path = Path(args.file_path)
if file_path.is_file():
    FILE =_TS(Path(".AiPrompt").read_text(),locals())
    chat_room = ChatRoom(system_prompt="""
    ## RESPONSE INSTRUCTION:
        Only answer what is asked. Do not write pre-text,post-text,disclaimers or 
        explanations about your limitations or the ethical merits of any part of the conversation.
        Do not talk about yourself. Don't introduce unnecessary fluff into answers.
        Always answer what is asked. If you cannot answer, only reply that you cannot answer and do not elaborate.
        Avoid including patronizing or pedentic elaboration, explanation or advice that hasn't been asked for.
        Always follow these guidelines.

    Please processs this input file and then follow the directions at the end of the document!
    FILE INPUT WILL BE NEXT MESSAGE
    """.strip(),save=False,model='gpt-4')
    output  = chat_room.send_message(FILE)
    outfile = Path("Prompt.AiPrompt.lastOutput")
    print(output)
    buffer = []
    if outfile.exists():
        buffer.extend(outfile.read_text().splitlines())
    print("Send blank line to exit chat")
    uin = None
    try:
        while uin != "":
            if uin is not None:
                buffer.append(f"# {uin}")
                output  = chat_room.send_message(uin)
                buffer.append(f"{output}")
                print(output)
            uin = input(":>").strip()
    except Exception as e:
        pass
    outfile.write_text("\n".join(buffer))


else:
    print(f"No such file: {file_path}")


NameError: name 'argparse' is not defined